## Proyecto: Turismo de los Alpes  - Grupo 34
Felipe Nuñez
<br>
Jeronimo Vargas Rendon - 202113305
<br>
Estudiante 1: Juan Manuel Pérez - 202021827

## 1. Entendimiento de los datos

## 2. Preparación de los datos

## 3. Modelo

## 4. Evaluacion

## 5. Resultados